In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/nlp_sentiment/

/content/gdrive/MyDrive/nlp_sentiment


In [4]:
%cd  /usr/local/lib/python3.7/dist-packages/

/usr/local/lib/python3.7/dist-packages


In [5]:
%ls

absl/
absl_py-0.12.0.dist-info/
alabaster/
alabaster-0.7.12.dist-info/
albumentations/
albumentations-0.1.12.dist-info/
altair/
altair-4.1.0.dist-info/
apiclient/
appdirs-1.4.4.dist-info/
appdirs.py
apt/
apt_inst.cpython-37m-x86_64-linux-gnu.so*
apt_inst.pyi
apt_pkg.cpython-37m-x86_64-linux-gnu.so*
apt_pkg.pyi
aptsources/
argon2/
argon2_cffi-20.1.0.dist-info/
astor/
astor-0.8.1.dist-info/
astropy/
astropy-4.2.dist-info/
astunparse/
astunparse-1.6.3.dist-info/
async_generator/
async_generator-1.10.dist-info/
atari_py/
atari_py-0.2.6.dist-info/
atomicwrites/
atomicwrites-1.4.0.dist-info/
attr/
attrs-20.3.0.dist-info/
audioread/
audioread-2.1.9.dist-info/
autograd/
autograd-1.3.dist-info/
babel/
Babel-2.9.0.dist-info/
backcall/
backcall-0.2.0.dist-info/
beautifulsoup4-4.6.3.dist-info/
bin/
bleach/
bleach-3.3.0.dist-info/
blis/
blis-0.4.1.dist-info/
bokeh/
bokeh-2.3.0.dist-info/
bottleneck/
Bottleneck-1.3.2.dist-info/
branca/
branca-0.4.2.dist-info/
bs4/
bs4-0.0.1.dist-info/
bson/
cachecon

In [6]:
%rm -rf torch*

In [7]:
%ls

absl/
absl_py-0.12.0.dist-info/
alabaster/
alabaster-0.7.12.dist-info/
albumentations/
albumentations-0.1.12.dist-info/
altair/
altair-4.1.0.dist-info/
apiclient/
appdirs-1.4.4.dist-info/
appdirs.py
apt/
apt_inst.cpython-37m-x86_64-linux-gnu.so*
apt_inst.pyi
apt_pkg.cpython-37m-x86_64-linux-gnu.so*
apt_pkg.pyi
aptsources/
argon2/
argon2_cffi-20.1.0.dist-info/
astor/
astor-0.8.1.dist-info/
astropy/
astropy-4.2.dist-info/
astunparse/
astunparse-1.6.3.dist-info/
async_generator/
async_generator-1.10.dist-info/
atari_py/
atari_py-0.2.6.dist-info/
atomicwrites/
atomicwrites-1.4.0.dist-info/
attr/
attrs-20.3.0.dist-info/
audioread/
audioread-2.1.9.dist-info/
autograd/
autograd-1.3.dist-info/
babel/
Babel-2.9.0.dist-info/
backcall/
backcall-0.2.0.dist-info/
beautifulsoup4-4.6.3.dist-info/
bin/
bleach/
bleach-3.3.0.dist-info/
blis/
blis-0.4.1.dist-info/
bokeh/
bokeh-2.3.0.dist-info/
bottleneck/
Bottleneck-1.3.2.dist-info/
branca/
branca-0.4.2.dist-info/
bs4/
bs4-0.0.1.dist-info/
bson/
cachecon

In [8]:
!pip install torch==1.7.0
!pip install torchtext==0.8.0

     |████████████████████████████████| 776.8MB 22kB/s 
ERROR: fastai 1.0.61 requires torchvision, which is not installed.
     |████████████████████████████████| 7.0MB 7.8MB/s 


In [9]:
import torch
print(torch.__version__)
import torch.nn as nn
from torch import optim
import torchtext
print(torchtext.__version__)
from torchtext.data import Field,TabularDataset,BucketIterator

torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.backends.cudnn.enabled=False
torch.backends.cudnn.deterministic=True

1.7.0
0.8.0


In [10]:
class TweetSentimentDataLoader(object) :
  ## csv_file : 데이터를 담고 있는 csv 파일의 경로 , string 
  def __init__(self, train_fn, batch_size=64, field_nm=None, valid_ratio=.2, device=-1, max_vocab=99999990, min_freq=1, use_eos=False, shuffle=True) :
    super().__init__()

    # sequential : 데이터 타입이 Sequential한지 여부를 설정함. False이면 tokenization이 수행되지 않음. 
    #              레이블은 Sequential 데이터 타입이 아니기 때문에 False로 둠
    # use_vocab : Vocab Object를 사용할지 여부를 설정함. False이면 해당 Field의 데이터는 숫자형 데이터 
    #             여야함. 여기서는 레이블이 문자형 데이터이므로 숫자형 데이터로 변환하기 위해 True로 함 
    # unk_token : OOV에 대응하는 단어를 나타내기 위해 <unk> token을 사용할지 여부를 설정함. 여기서는 
    #             None으로 설정(레이블에는 실제로 OOV에 대응하는 단어가 존재하지 않는다고 가정) 
    self.label = Field(
        sequential=False,
        use_vocab=True,
        unk_token=None
    )

    # sequential : 입력 데이터는 자연어이므로 Sequential하기 떄문에 디폴트 값인 True를 사용함 
    # use_vocab : 입력 데이터가 자연어이므로 False 
    # batch_first : batch에 대응하는 차원을 가장 앞으로 둘지 여부를 설정함. 여기서는 생각하기 편하게 
    #               batch에 대응하는 차원을 가장 앞에 둠 
    # include_lengths : padded minibatch만 리턴받을지 아나면 길이까지 같이 리턴받을지를 설정함.
    #                   여기서는 padded minibatch만 리턴받음  
    # eos_token : 모든 sample의 끝에 지정한 token을 붙일지 여부를 설정. 
    self.text = Field(
        use_vocab=True,
        batch_first=True,
        include_lengths=False,
        eos_token='<EOS>' if use_eos else None
    )

    # csv 파일에서 데이터를 읽어들임 
    self.tabular_dataset = TabularDataset(
        path=train_fn,
        format='csv',
        fields=[
                (field_nm[0], self.label),
                (field_nm[1], self.text)
        ],
    )
    if valid_ratio > 0: 
      train, valid =  self.tabular_dataset.split(1-valid_ratio)
     

      # BucketIterator : 하나의 batch가 최대한 비슷한 길이의 sample로 구성될 수 있도록 해주는 dataloader 
      #                  sample에 추가되는 padding의 숫자를 최소화함. 
      self.train_loader, self.valid_loader = BucketIterator.splits(
          (train,valid),
          batch_size=batch_size,
          device='cuda:%d' % device if device>=0 else 'cpu',
          shuffle=shuffle,
          sort_key = lambda x: len(x.cleaned_text),
          sort_within_batch=True 
      )
    else :
      train = self.tabular_dataset
      self.train_loader = BucketIterator(
          train,
          batch_size=batch_size,
          device = 'cuda:%d' %device if device>=0 else 'cpu',
          shuffle=shuffle,
          sort_key= lambda x: len(x.cleaned_text),
          sort_within_batch=True
      )

    # data에 대한 vocabulary set을 생성 (단어를 특정 숫자에 mapping 해줌 -> model에 입력되기 위해서는 숫자가 될 필요가 있음)
    # 해당 field에 나타날 수 있는 값들로 이루어진 dataset object 또는 다른 iterable datasource 
    # max_size = vocabulary set에 들어갈 수 있는 단어의 최대 수 
    # min_freq = 특정 단어가 vocabulary set에 들어가기 위해 필요한 조건으로서의 출현 빈도 
    self.label.build_vocab(train)
    self.text.build_vocab(train, max_size=max_vocab, min_freq=min_freq)


In [11]:
%cd /content/gdrive/MyDrive/

/content/gdrive/MyDrive


In [12]:
%cd /content/gdrive/MyDrive/nlp_sentiment/
tw_senti_dl = TweetSentimentDataLoader('./augmented_cleaned_train.csv',field_nm=['sentiment','cleaned_text'],valid_ratio=0.1, device=0,min_freq=5)

/content/gdrive/MyDrive/nlp_sentiment


/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warn

In [13]:
#Hyper Parameters

n_epochs = 20
input_size = len(tw_senti_dl.text.vocab)
word_vec_size = 256
hidden_size =  1024
n_classes = len(tw_senti_dl.label.vocab)
dropout_p = 0.3

n_layers = 4

learning_rate = 0.00001
betas = (0.9,0.99)

In [14]:
print(input_size)

25716


In [15]:
class LSTMClassifier(nn.Module) :

  def __init__(self, input_size, word_vec_size, hidden_size, n_classes, n_layers=4, dropout_p=.3, batch_norm=False) :
    self.input_size = input_size
    self.word_vec_size = word_vec_size
    self.hidden_size = hidden_size
    self.n_classes = n_classes 
    self.n_layers = n_layers
    self.dropout_p = dropout_p
    self.batch_norm = batch_norm
    
    super().__init__()

    # Embedding Layer를 만들어 훈련 데이터로부터 처음부터 임베딩 벡터를 학습하는 방법
    # num_embeddings : Embedding을 할 단어들의 개수. 단어 집합의 개수 
    # embedding_dim : Embedding할 벡터의 차원. 하이퍼 파라미터   
    self.emb = nn.Embedding(num_embeddings = input_size, embedding_dim = word_vec_size)

    # input_size : Neural Net의 Input Vector의 크기. Embedding을 거치므로 Embedding Vector의 차원과 동일 
    # hidden_size : RNN의 Hidden Vector의 크기 
    # num_layers : RNN의 Layer의 개수 
    # dropout : Dropout할 비율(Node 중 랜덤하게 떨어뜨릴 노드의 비율)
    # batch_first : Neural Net의 Output Vector의 가장 첫번째 차원을 batch index에 대한 차원으로 설정 
    # bidirectional : 양방향으로 진행됨 
    self.rnn_list = nn.ModuleList()
    if batch_norm :
      self.norm_list = nn.ModuleList()
    
    for i in range(n_layers) :
      if i==0 :
        cur_input_size = word_vec_size 
      else : 
        cur_input_size = 2*hidden_size
      cur_layer = nn.LSTM(
          input_size=cur_input_size,
          hidden_size = hidden_size,
          dropout = dropout_p,
          batch_first=True, 
          bidirectional=True,
        )
      self.rnn_list.append(cur_layer)
      if batch_norm :
        cur_norm = nn.BatchNorm1d(2*hidden_size,affine=False)
        self.norm_list.append(cur_norm)
    

    ## Neural Net의 Output을 기반으로 input이 임의의 class에 대하여 특정 class일 확률을 구함
    ## in_features : FC Layer의 input vector의 크기(여기서는 RNN의 output vector의 크기)
    ## out_features : FC_Layer의 output vector의 크기(Classification을 위한 Neural Network이므로 Class의 개수와 같음) 
    self.output_fc = nn.Linear(in_features = hidden_size*2, out_features = n_classes)
    
    ## softmax vs logsoftmax 
    ## softmax를 사용하면 vanishing gradients 문제에 취약해짐 -> logsoftmax를 통해 해결 
    self.log_soft_max = nn.LogSoftmax(dim=-1)


  def forward(self, x) :
    #|x| = (batch_size, length=embed_vec_num = word_num)
    x = self.emb(x)

    #|x| = (batch_size, length=embed_vec_num = word_num, word_vec_size=embed_vec_size)
    #x, _ = self.rnn(x)
    for i in range(self.n_layers) :
      x,_ = self.rnn_list[i](x)
      if self.batch_norm :
        x = x.permute([0,2,1])
        x = self.norm_list[i](x)
        x = x.permute([0,2,1])
 
    #|x| = (batch_size, length=embed_vec_num = word_num, hidden_size*2)
    #x[:,-1] -> RNN의 여러 output vector중 마지막 output vector만 사용 
    y = self.log_soft_max(self.output_fc(x[:,-1]))

    #|y| = (batch_size, n_classes)
    return y

In [16]:
class RNNClassifier(nn.Module) :

  def __init__(self, input_size, word_vec_size, hidden_size, n_classes, n_layers=4, dropout_p=.3, batch_norm=False) :
    self.input_size = input_size
    self.word_vec_size = word_vec_size
    self.hidden_size = hidden_size
    self.n_classes = n_classes 
    self.n_layers = n_layers
    self.dropout_p = dropout_p
    self.batch_norm = batch_norm
    
    super().__init__()

    # Embedding Layer를 만들어 훈련 데이터로부터 처음부터 임베딩 벡터를 학습하는 방법
    # num_embeddings : Embedding을 할 단어들의 개수. 단어 집합의 개수 
    # embedding_dim : Embedding할 벡터의 차원. 하이퍼 파라미터   
    self.emb = nn.Embedding(num_embeddings = input_size, embedding_dim = word_vec_size)

    # input_size : Neural Net의 Input Vector의 크기. Embedding을 거치므로 Embedding Vector의 차원과 동일 
    # hidden_size : RNN의 Hidden Vector의 크기 
    # num_layers : RNN의 Layer의 개수 
    # dropout : Dropout할 비율(Node 중 랜덤하게 떨어뜨릴 노드의 비율)
    # batch_first : Neural Net의 Output Vector의 가장 첫번째 차원을 batch index에 대한 차원으로 설정 
    # bidirectional : 양방향으로 진행됨 
    self.rnn_list = nn.ModuleList()
    if batch_norm :
      self.norm_list = nn.ModuleList()
    
    for i in range(n_layers) :
      if i==0 :
        cur_input_size = word_vec_size 
      else : 
        cur_input_size = 2*hidden_size
      cur_layer = nn.RNN(
          input_size=cur_input_size,
          hidden_size = hidden_size,
          nonlinearity='relu',
          dropout = dropout_p,
          batch_first=True, 
          bidirectional=True,
        )
      self.rnn_list.append(cur_layer)
      if batch_norm :
        cur_norm = nn.BatchNorm1d(2*hidden_size)
        self.norm_list.append(cur_norm)


    ## Neural Net의 Output을 기반으로 input이 임의의 class에 대하여 특정 class일 확률을 구함
    ## in_features : FC Layer의 input vector의 크기(여기서는 RNN의 output vector의 크기)
    ## out_features : FC_Layer의 output vector의 크기(Classification을 위한 Neural Network이므로 Class의 개수와 같음) 
    self.output_fc = nn.Linear(in_features = hidden_size*2, out_features = n_classes)  

    ## softmax vs logsoftmax 
    ## softmax를 사용하면 vanishing gradients 문제에 취약해짐 -> logsoftmax를 통해 해결 
    self.log_soft_max = nn.LogSoftmax(dim=-1)


  def forward(self, x) :
    #|x| = (batch_size, length=embed_vec_num = word_num)
    x = self.emb(x)

    #|x| = (batch_size, length=embed_vec_num = word_num, word_vec_size=embed_vec_size)
    #x, _ = self.rnn(x)
    for i in range(self.n_layers) :
      x,_ = self.rnn_list[i](x)
      if self.batch_norm :
        x = x.permute([0,2,1])
        x = self.norm_list[i](x)
        x = x.permute([0,2,1])
 
    #|x| = (batch_size, length=embed_vec_num = word_num, hidden_size*2)
    #x[:,-1] -> RNN의 여러 output vector중 마지막 output vector만 사용 
    y = self.log_soft_max(self.output_fc(x[:,-1]))

    #|y| = (batch_size, n_classes)
    return y

In [17]:
class LabelSmoothingLoss(nn.Module) :
  def __init__(self,num_classes, smoothing=0.1) :
    self.num_classes = num_classes
    self.smoothing = smoothing 

  def __call__(self, pred, target) :
    # | target | = (batch size, )
    # | index | = (index, 1) 

    # for one hot encoding 
    index = target.view((target.size()[0],1)).to(0)
    src = torch.ones((target.size()[0],1)).to(0)
    one_hot = torch.zeros((target.size()[0], self.num_classes)).to(0).scatter_(1, index, src)

    
    one_hot = one_hot*(1-self.smoothing) + self.smoothing/self.num_classes

    # for nll loss with float preds (pred is log likelihood)
    nll_loss = ((-pred) * one_hot).mean()

    return nll_loss



In [18]:
#LSTMClassifier
tw_senti_classifier = LSTMClassifier(input_size, word_vec_size, hidden_size, n_classes, n_layers=n_layers, dropout_p=dropout_p)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [19]:
#Loss Function 
#smooth_loss = LabelSmoothingLoss(n_classes,0.0)
smooth_loss = LabelSmoothingLoss(n_classes,smoothing=0.2)
#smooth_loss = nn.NLLLoss()

#Optimizer : Adam Optimizer 
optimizer = optim.Adam(tw_senti_classifier.parameters(), lr=learning_rate, )

In [20]:
#To GPU 
tw_senti_classifier.cuda(0)
#smooth_loss.cuda(0)

LSTMClassifier(
  (emb): Embedding(25716, 256)
  (rnn_list): ModuleList(
    (0): LSTM(256, 1024, batch_first=True, dropout=0.3, bidirectional=True)
    (1): LSTM(2048, 1024, batch_first=True, dropout=0.3, bidirectional=True)
    (2): LSTM(2048, 1024, batch_first=True, dropout=0.3, bidirectional=True)
    (3): LSTM(2048, 1024, batch_first=True, dropout=0.3, bidirectional=True)
  )
  (output_fc): Linear(in_features=2048, out_features=4, bias=True)
  (log_soft_max): LogSoftmax(dim=-1)
)

In [21]:
train_loss_mean = []
train_acc_mean = []

valid_loss_mean = []
valid_acc_mean = []

## Training, Validation 
for i in range(n_epochs):
  loss_sum = 0.0
  accuracy_sum = 0.0
  batch_len = 0
  for id, cur_batch in enumerate(tw_senti_dl.train_loader) :
    ##Feed Forward
    log_probs = tw_senti_classifier.forward(cur_batch.cleaned_text)
    #log_probs = tw_senti_classifier2.forward(cur_batch.cleaned_text)    

    #log_probs = (log_probs +log_probs2)/2

    #Calculate Loss
    loss = smooth_loss(log_probs, cur_batch.sentiment)

    #Backpropagation 
    loss.backward()

    #Gradient Descent 
    optimizer.step()

    #Calculate Accuracy 
    acc = (torch.argmax(log_probs, dim=-1) == cur_batch.sentiment).sum() / float(cur_batch.sentiment.size(0))

    loss_sum += float(loss)
    accuracy_sum += float(acc)
    batch_len+=1

    #print training result per mini batch
    print(f'Train Epoch #{i+1}, Batch {id+1}/{len(tw_senti_dl.train_loader)} loss = {float(loss)}, accuracy = {float(acc)}')
  #print training result per epoch
  print(f'Train Epoch #{i+1} loss_mean = {loss_sum/batch_len}, accuracy_mean={accuracy_sum/batch_len}')
  print('='*300)    
  train_loss_mean.append(loss_sum/batch_len)
  train_acc_mean.append(accuracy_sum/batch_len)

  loss_sum = 0.0
  accuracy_sum = 0.0
  batch_len = 0
  with torch.no_grad() :
    for id, cur_batch in enumerate(tw_senti_dl.valid_loader) :    
      log_probs = tw_senti_classifier.forward(cur_batch.cleaned_text)
      #log_probs = tw_senti_classifier2.forward(cur_batch.cleaned_text)
      #log_probs = (log_probs +log_probs2)/2
      loss = smooth_loss(log_probs, cur_batch.sentiment)
      acc = (torch.argmax(log_probs, dim=-1) == cur_batch.sentiment).sum() / float(cur_batch.sentiment.size(0))
      loss_sum += float(loss)
      accuracy_sum += float(acc)
      batch_len += 1
      print(f'Test Epoch #{i+1}, Batch {id+1}/{len(tw_senti_dl.valid_loader)} loss = {float(loss)}, accuracy = {float(acc)}')
  print(f'Test Epoch #{i+1} loss_mean = {loss_sum/batch_len}, accuracy_mean={accuracy_sum/batch_len}')
  print('='*300)  
  valid_loss_mean.append(loss_sum/batch_len)
  valid_acc_mean.append(accuracy_sum/batch_len)



torch.cuda.empty_cache()

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Train Epoch #19, Batch 3459/3802 loss = 0.15831664204597473, accuracy = 0.96875
Train Epoch #19, Batch 3460/3802 loss = 0.1474452167749405, accuracy = 1.0
Train Epoch #19, Batch 3461/3802 loss = 0.1779344230890274, accuracy = 0.890625
Train Epoch #19, Batch 3462/3802 loss = 0.15194398164749146, accuracy = 0.984375
Train Epoch #19, Batch 3463/3802 loss = 0.19033710658550262, accuracy = 0.859375
Train Epoch #19, Batch 3464/3802 loss = 0.16340765357017517, accuracy = 0.96875
Train Epoch #19, Batch 3465/3802 loss = 0.15659520030021667, accuracy = 0.984375
Train Epoch #19, Batch 3466/3802 loss = 0.15322178602218628, accuracy = 0.984375
Train Epoch #19, Batch 3467/3802 loss = 0.18612799048423767, accuracy = 0.890625
Train Epoch #19, Batch 3468/3802 loss = 0.15588562190532684, accuracy = 0.984375
Train Epoch #19, Batch 3469/3802 loss = 0.1471298336982727, accuracy = 1.0
Train Epoch #19, Batch 3470/3802 loss = 0.15721836686134338, accuracy = 0.984375
Train E

In [ ]:
print(train_loss_mean)
print(train_acc_mean)

[0.2672235862102589, 0.20646617630950279, 0.17229352166767437, 0.16060351307206502, 0.15647088148401386, 0.1548447632149983, 0.154114670212773, 0.15360916875839736, 0.1533612303400529, 0.15333542324159724, 0.15326024763979829, 0.15334536464025572, 0.1532904800499884, 0.15348013108699088, 0.15362337441318352, 0.15401129671487604, 0.15421201722174177, 0.15466823089850193, 0.15519035829408492, 0.15580752029866057]
[0.6340865005260389, 0.8424924381904261, 0.9366616254602841, 0.9664855668069438, 0.9760898869016307, 0.9801707982640715, 0.9819585086796423, 0.9824187927406628, 0.9830475736454498, 0.9829407219884272, 0.9828831864807995, 0.9826283863755918, 0.9827681154655444, 0.9822996120462915, 0.9819954957916885, 0.9810461599158338, 0.9803187467122567, 0.9792379011046818, 0.9779721199368754, 0.9760816675433982]


In [ ]:
print(valid_loss_mean)
print(valid_acc_mean)

[0.23680420364330282, 0.18956809936544855, 0.17412575709481612, 0.16948572977778478, 0.16792740420951346, 0.16839518966967895, 0.16894302378027715, 0.1686418027249352, 0.16996743898160632, 0.16997119551854775, 0.17082451611545915, 0.1714851091446324, 0.1710881094490101, 0.17165723125985327, 0.1718088494829534, 0.17190568529544992, 0.1724593635851046, 0.17258296554674776, 0.17315297353634035, 0.17382908850274187]
[0.7519240158105854, 0.8902186761229315, 0.9297798463356974, 0.9428271790768238, 0.9482860520094563, 0.9482121749408984, 0.9473995271867612, 0.9487662529550828, 0.9473625886524822, 0.9461066784869976, 0.9439642434988179, 0.9428930260047281, 0.9436028882966819, 0.9440381205673759, 0.9427083333333334, 0.9419695626477541, 0.9416001773049646, 0.9405658983451537, 0.9375, 0.9362071513002365]


a

In [ ]:
### 테스트 
